In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
smtp_server = SMTPTestServer(email_server)
smtp_server.start()

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
high_client.worker_pools

## DO tests the API endpoints

In [ ]:
high_client.custom_api.api_endpoints()

In [ ]:
assert len(high_client.custom_api.api_endpoints()) == 3

In [ ]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

In [ ]:
# stdlib
import os

mock_call_per_min = int(os.environ.get("MOCK_CALL_PER_MIN", 5))
mock_call_per_min

### Test the `test_query` endpoint

In [ ]:
# Test mock version
for _ in range(mock_call_per_min - 4):
    result = high_client.api.services.bigquery.test_query.mock(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
    )
    assert len(result) == 10

In [ ]:
# Test mock version for wrong queries
with sy.raises(
    sy.SyftException(public_message="*must be qualified with a dataset*"), show=True
):
    _ = high_client.api.services.bigquery.test_query.mock(
        sql_query="SELECT * FROM invalid_table LIMIT 1"
    )

In [ ]:
# Test query timed out
with sy.raises(sy.SyftException(public_message="*timed out.*"), show=True):
    _ = high_client.api.services.bigquery.test_query.mock(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} TIMED OUT"
    )

In [ ]:
# Test too big query
with sy.raises(sy.SyftException(public_message="*exhausts all resources.*"), show=True):
    _ = high_client.api.services.bigquery.test_query.mock(
        sql_query="SELECT TOO BIG QUERY"
    )

In [ ]:
# Test mock version: rate limit has been reached
with sy.raises(
    sy.SyftException(
        public_message="*Rate limit of calls per minute has been reached.*"
    ),
    show=True,
):
    _ = high_client.api.services.bigquery.test_query.mock(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
    )

In [ ]:
state = high_client.api.services.bigquery.test_query.mock.context.state
state

In [ ]:
assert len(state[ADMIN_EMAIL]) == mock_call_per_min

In [ ]:
# Test private version
result = high_client.api.services.bigquery.test_query.private(
    sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
)
result

### Test the `schema` endpoint

In [ ]:
# Testing schema
result = high_client.api.services.bigquery.schema()
result

In [ ]:
assert len(result) == 23

### Test the `submit_query` endpoint

In [ ]:
result = high_client.api.services.bigquery.submit_query(
    func_name="my_func",
    query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 1",
)

In [ ]:
result

In [ ]:
assert "Query submitted" in result

In [ ]:
email_server.get_emails_for_user(user_email=ADMIN_EMAIL)

In [ ]:
email_server.get_emails_for_user(user_email="admin@bigquery.org")

In [ ]:
assert len(email_server.get_emails_for_user(user_email="admin@bigquery.org")) > 0

## DS tests the API endpoints

In [ ]:
DS_EMAIL, DS_PW = "data_scientist@openmined.org", "verysecurepassword"

res = high_client.register(
    name="data_scientist",
    email=DS_EMAIL,
    password=DS_PW,
    password_verify=DS_PW,
)
res

In [ ]:
high_ds_client = sy.login(url="http://localhost:8080", email=DS_EMAIL, password=DS_PW)

In [ ]:
high_ds_client.custom_api.api_endpoints()

In [ ]:
assert len(high_ds_client.custom_api.api_endpoints()) == 3

### DS tests the `test_query` endpoint

In [ ]:
# DS can't access the private test_query endpoint
with sy.raises(
    sy.SyftException(public_message="*You're not allowed to run this code.*"), show=True
):
    _ = high_ds_client.api.services.bigquery.test_query.private(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
    )

In [ ]:
# Test mock version
for _ in range(mock_call_per_min - 3):
    result = high_ds_client.api.services.bigquery.test_query.mock(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
    )
    assert len(result) == 10

In [ ]:
# Test mock version for wrong queries
with sy.raises(
    sy.SyftException(
        public_message="*something went wrong during this endpoint execution, please contact your admin.*"
    ),
    show=True,
):
    _ = high_ds_client.api.services.bigquery.test_query.mock(
        sql_query="SELECT * FROM invalid_table LIMIT 1"
    )

In [ ]:
# Test query timed out
with sy.raises(
    sy.SyftException(
        public_message="*something went wrong during this endpoint execution, please contact your admin.*"
    ),
    show=True,
):
    _ = high_ds_client.api.services.bigquery.test_query.mock(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} TIMED OUT"
    )

In [ ]:
# Test mock version: rate limit has been reached
with sy.raises(
    sy.SyftException(
        public_message="*something went wrong during this endpoint execution, please contact your admin.*"
    ),
    show=True,
):
    _ = high_ds_client.api.services.bigquery.test_query.mock(
        sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
    )

In [ ]:
state = high_client.api.services.bigquery.test_query.mock.context.state
state

In [ ]:
assert len(state[DS_EMAIL]) == mock_call_per_min

## Land the server

In [ ]:
smtp_server.stop()

In [ ]:
server.land()